Alcuni blocchi di codice non li ho rieseguiti in quanto ho utilizzato i dataset puliti precedentemente, inoltre ho lavorato su jupiter notebook perchè colab dava qualche problemino con le operazio di lunga durata.

In [ ]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from tqdm import tqdm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')
import gensim.corpora as corpora
import spacy
import string
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
fake_news = pd.read_csv("/Fake.csv")
true_news = pd.read_csv("/True.csv")

In [ ]:
fake_news # testo se carica il dataset

In [ ]:
true_news # testo se carica il dataset

In [ ]:
counts_cat_fake = fake_news['subject'].value_counts()
max_category = counts_cat_fake.idxmax()
frequency = counts_cat_fake.max()


In [ ]:
dataset_fake = fake_news['text'] #prendo solo le colonne da pulire
dataset_true = true_news['text']

In [ ]:
#### DATA CLEANING

In [ ]:
english_stopwords = stopwords.words('english')
nlp = spacy.load( 'en_core_web_sm')
punctuation = set(string.punctuation)

def data_cleaner (sentence):
  sentence = sentence.lower()
  for c in string.punctuation:
    sentence = sentence.replace(c, ' ')

  document = nlp(sentence)
  sentence = " ".join(token.lemma_ for token in document)
  sentence = " ".join(word for word in sentence.split() if word not in english_stopwords)
  sentence = re.sub('\d','',sentence)
  sentence = re.sub(' +',' ',sentence)


  return sentence

In [ ]:
print(f"Le fake news sono più frequenti nella categoria: {max_category} con {frequency} fakenews ")

Le fake news sono più frequenti nella categoria: News con 9050 fakenews 


In [ ]:
dataset_fake_cleaned = [data_cleaner(document) for document in dataset_fake ]


In [ ]:
data_cleaner(dataset_fake_cleaned[2001])#per testare

In [ ]:

#creo il dataframe da salvare su disco
dataframe = pd.DataFrame({'text':dataset_fake_cleaned})
dataframe.to_csv('./fake_cleaned.csv',index=False)


In [ ]:
#eseguo la funzione anche sul dataset di news reali
dataset_true_cleaned = [data_cleaner(document) for document in dataset_true ]


In [ ]:
#creo il dataframe da salvare su disco
dataframe_true = pd.DataFrame({'text':dataset_true_cleaned})
dataframe_true.to_csv('./true_cleaned.csv',index=False)

In [ ]:
#prendo solo le colonne text da passare al data cleaner
fake_news_clean = pd.read_csv("fake_cleaned.csv")
true_news_clean = pd.read_csv("true_cleaned.csv")

In [ ]:
documents = fake_news_clean['text']

In [ ]:
# funzione per il preprocessing
def sent_to_word(items):
    for item in items:
        if isinstance(item, str):  # Controlla che sia una stringa
            yield simple_preprocess(item, deacc=True)
        elif isinstance(item, float) and pd.isna(item):  # Se è NaN, ignora
            continue
        else:
            yield simple_preprocess(str(item), deacc=True)  # Converte in stringa altri tipi

def remove_stopwords(texts):
  return [[word for word in words if word not in stop_words and len(word) >=5] for words in texts]

data_words = list(sent_to_word(documents))
data_words = remove_stopwords(data_words)

In [ ]:
# creo il bag of words
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]

In [ ]:
# questo blocco mi restituisce i 10 topic che poi confrontero con tutti i documents
from pprint import pprint
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes = 3
                                      )
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
##test per ciclare la test_sentence con tutti i topic
test_sentence = "matter donald trump go never actually right thing fulfil duty potus one might think suffer monstrous fail american health care act president might double get right back work quest somehow improve obamacare"
document = id2word.doc2bow(simple_preprocess(test_sentence))
for index, score in lda_model[document]:
    print("TOPIC:" + str(index))
    print("SCORE:" + str(score))

In [ ]:
# calcolo il topic dominante
from collections import Counter

# Ottenere il topic dominante per ogni documento
doc_topics = [max(lda_model.get_document_topics(doc), key=lambda x: x[1])[0] for doc in corpus]

# Contare la frequenza di ogni topic
topic_counts = Counter(doc_topics)

# Trovare il topic più frequente
most_frequent_topic = topic_counts.most_common(1)[0]

# Creare un DataFrame per visualizzare la distribuzione dei topic
df_topic_counts = pd.DataFrame(topic_counts.items(), columns=["Topic", "Frequenza"]).sort_values(by="Frequenza", ascending=False)

# Restituire il topic più frequente
most_frequent_topic


In [ ]:
#Esistono argomenti che sono più soggetti alle fake news?
print(f"L'argomento più frequente è con alta probabilità: Topic {most_frequent_topic[0]} simile a {most_frequent_topic[1]} documenti ")

L'argomento più frequente è con alta probabilità: Topic 9 simile a 7228 documenti 


In [ ]:
#prendo solo la colonna title per passarla al data cleaner
title_fake = fake_news['title']
title_true = true_news['title']

In [ ]:
###cleaning colonna title fake
title_fake_cleaned = [data_cleaner(document) for document in title_fake ]

In [ ]:
###cleaning colonna title reali
title_true_cleaned = [data_cleaner(document) for document in title_true ]

In [ ]:
#salvo il dataset per non ripetere in seguito il data_cleaner
df_clean_title = pd.DataFrame({'text':dataset_fake_cleaned,'title':title_fake_cleaned})
df_clean_title.to_csv('./fake_cleaned.csv',index=False)

In [ ]:
df_clean_title_true = pd.DataFrame({'text':dataset_true_cleaned,'title':title_true_cleaned})
df_clean_title_true.to_csv('./true_cleaned.csv',index=False)

In [ ]:
#illustro tramite worldcloud il pattern più comune

from collections import Counter
from itertools import chain
from wordcloud import WordCloud

#ri-eseguo la cleaning dei titoli
df_clean_title['title'] = df_clean_title['title'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', str(x)))


# Tokenizzazione: separare le parole nei titoli
words_title = list(chain.from_iterable(df_clean_title['title'].str.split()))

# Contare le parole più frequenti
word_freq_title = Counter(words_title)

# Mostrare le 20 parole più comuni
common_words = word_freq_title.most_common(20)

# Generare una WordCloud per una visualizzazione alternativa
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq_title)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud delle parole più frequenti nei titoli")
plt.show()

# Restituire le parole più comuni
common_words


In [ ]:
#### in alternativa possiamo analizzare i titoli resi più accattivanti da determinate frasi strutturate
patterns = [
    "shocking",        # Parole sensazionalistiche
    "breaking",        # Breaking news
    "the truth about", # Frasi tipiche delle fake news
    "must see",        # Clickbait
    "exclusive",
    "urgent",
    "must see"
]

# Conta le occorrenze dei pattern nei titoli
pattern_counts = {p: df_clean_title['title'].str.contains(p, regex=True).sum() for p in patterns}

print(pattern_counts)



{'shocking': 208, 'breaking': 15, 'the truth about': 0, 'must see': 15, 'exclusive': 22, 'urgent': 7}


In [ ]:
#pattern legati alla punteggiatura, prenderò la colonna senza passarla al data_cleaner
import re

# Creo nuove colonne per contare gli elementi di punteggiatura nei titoli delle fake news
fake_news["exclamation_marks"] = fake_news["title"].astype(str).apply(lambda x: len(re.findall(r'!', x)))
fake_news["question_marks"] = fake_news["title"].astype(str).apply(lambda x: len(re.findall(r'\?', x)))
fake_news["multiple_punctuation"] = fake_news["title"].astype(str).apply(lambda x: len(re.findall(r'[!?]{2,}', x)))
fake_news["quotes"] = fake_news["title"].astype(str).apply(lambda x: len(re.findall(r'"', x)))
fake_news["all_caps"] = fake_news["title"].astype(str).apply(lambda x: 1 if re.search(r'\b[A-Z]{4,}\b', x) else 0)

# Visualizzare le statistiche
fake_news[["exclamation_marks", "question_marks", "multiple_punctuation", "quotes", "all_caps"]].sum()


exclamation_marks        3606
question_marks           1740
multiple_punctuation       28
quotes                      0
all_caps                18689
dtype: int64

In [ ]:
#re-importo i modelli per passare alla fase addestramento
fake_news = pd.read_csv("/Fake.csv")
true_news = pd.read_csv("/True.csv")

In [ ]:
#aggiungo la colonna label identificando 0 Fake 1 Real e unisco i 2 dataset
fake_news['label']=0
true_news['label']=1
df_concat = pd.concat([fake_news, true_news], ignore_index=True)
df_concat

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [ ]:
#suddivido il dataset in parti test e train
c, text, c1, label = train_test_split(df_concat['text'],df_concat['label'], test_size=0.10, random_state=42)

In [ ]:
#verifico se è equilibrato
print(len(label[label==0]))
print(len(label[label==1]))

2384
2106


In [ ]:
#passo le notizie al data_cleaner
news_cleaned = []
for t in text :
    news_cleaned.append(data_cleaner(t))

In [ ]:
#prendo il 20% del dataset e lo passo al train test
x,x_test,y,y_test = train_test_split(news_cleaned, label, test_size = 0.2, random_state=56)

In [ ]:
#inizializzo il CountVectorizer
vec = CountVectorizer()
x = vec.fit_transform(x).toarray()

In [ ]:
#passo la parte test al vettorizzatore
x_test = vec.transform(x_test).toarray()

In [ ]:
#inizializzo
model = MultinomialNB()
model.fit(x,y)

MultinomialNB()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
#testo il modell con frasi esterne
test_news = "BREAKING NEWS!!!  I am a new president"
test_news_clean = data_cleaner(test_news)
test_news_cv = vec.transform([test_news_clean])
if model.predict(test_news_cv)==0:
    print("Probably fake_news")
else :
    print("Probably true_news")

Probably fake_news


In [ ]:
test_news1 = "WHO Reports Significant Decline in Global COVID-19 Cases "
test_news_clean1 = data_cleaner(test_news1)
test_news_cv1 = vec.transform([test_news_clean1])
if model.predict(test_news_cv1) == 0:
    print("Probably fake_news")
else:
    print("Probably true_news")

Probably true_news


In [ ]:
#verifico l'accuratezza
model.predict_proba(test_news_cv1)

array([[0.13592575, 0.86407425]])

In [ ]:
#calcolo delle metriche
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
#stampa dei risultati
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Accuracy: 0.95
Precision: 0.95
Recall: 0.95


In [ ]:
#esporto in formato pickle per l'implementazione.
import pickle

with open("model.pkl", "wb") as file:
    pickle.dump(model, file)